In [1]:
import datetime
import scipy as scp
from ribbonv2 import *
from samplingv4 import *
from plotResult_v2 import *
import numpy as np
import sklearn.metrics 
import pickle

import warnings

In [3]:
"""
load data
needed to take original stimulus
"""

filename = '../data/cell1.hdf5'
f = h5py.File(filename, 'r')

for item in f.attrs:
    print(item, f.attrs[item])
    
stim_binary = f['stim_binary'][:]
stim_gauss = f['stim_gauss'][:]

#r_binary = f['r_binary'][:]
#r_gauss = f['r_gauss'][:]

f.close()


# choose data
light = stim_binary
#data = r_binary


# stim time with time resolution of dt =1ms
stimT = np.arange(0, len(light),1)/1000


dt[ms] for r = 10
dt[ms] for stim = 1
info: Stim and vesicle release of one cell, 4 recordings per stim, binned sum for r but raw for stim (dt=1)


### produce data summary stats

In [4]:
"""
specify summary stats

for n>1
"""

#data=data[1:]

nSS = 9


## specify kernel to compare traces
g = scipy.signal.gaussian(10,2)

  
############################

"""
Pre run to get "true" data which we want to fit
"""



## define the gaussian kernel for the comparison of the traces
g = scipy.signal.gaussian(10,2) # (total width of kernel,std) # in 10*ms

# Define true parameters
trueK = 30#20
trueX0 = 0.8 #1.5#2#1.5
trueDockP = .1#05#.3
trueRibbonLambda = .4#.4
dockMax = 7
ribbonMax = 50
truerho = 0.3
truescale = 1 #0.5

params = [trueK,trueX0,trueDockP,trueRibbonLambda,dockMax,ribbonMax, truerho, truescale]

celltype = -1
light_ca_kernel =  - celltype * cone_kernel_scale(truescale)
stimCon = scp.signal.convolve(light, light_ca_kernel, mode="full")[:len(light)]  # /(1/dtstim *10)
# resample and normalize
dtstim = 0.001 # in ms
Ca_raw = resampleCon(stimCon, dtstim)
# normalize stim
stim = (Ca_raw - np.min(Ca_raw)) / np.max(np.abs(Ca_raw - np.min(Ca_raw)))

Ca = stim

## Compute 'True' Traces
nTrue = 4
data1 = runOne(params[:-1], Ca, correlated=True) #r_binary[0] # 
data = np.zeros((nTrue,len(data1)))

for i in range(0,nTrue):
    rel = runOne(params,  Ca, correlated=True) # r_binary[i] #
    data[i,:] = rel
###############################################



# number of recorded traces
nTrue = len(data)

lenData = len(data[0])
trueG = np.zeros((nTrue,lenData+len(g)-1))

for i in range(nTrue):
    trueG[i,:] = scipy.signal.convolve(data[i],g)
  

# specify weights for summary stats
unweighted_data_SS = makeSS(data,g,trueG, w = np.ones(nSS))
# normalizing factor
w_norm = 1/unweighted_data_SS
w_norm[np.isinf(w_norm)] = 1/48 #(1/48 mean value for the recorded traces)
#scaling factor for each sums stat
w_scale =  np.ones(len(unweighted_data_SS))
w_scale[0] = 1 # conv
w_scale[1] = 5 # sum of all
w_scale[2] = 5 # 1-fold
w_scale[3] = 5 # 2-fold
w_scale[4] = 2 # 4-fold
w_scale[5] = 2 # 5 fold
w_scale[6] = 4 # 6 fold
w_scale[7] = 1 # std

# final w 
w = w_norm * w_scale

dataSS = makeSS(data,g,trueG,w)



# fitting here

###### true parameters
trueK = 25#20
trueX0 = 0.8 #0.8 #1.5#2#1.5
trueDockP = .2#.3
trueRibbonLambda = .1#.4
dockMax = 8
ribbonMax = 50
truerho = 0.7

In [6]:
"""
specify priors here!!!
also: dock and ribbon size
"""

dockMax = 7
ribbonMax = 50

def makeHypersPrior():
    """
    :return: hyper parameters for the ribbon prior distributions
    """
    k_x0Hypers = [np.array([10, 0.5]), np.eye(2) * np.array([400, 0.1]), 4, 4]  # [mu0s,Lambda0, kappa0, nu0]
    dPHypers = [0.3, 0.05, 3, 3]  # [mu0, sigma0sq, kappa0, nu0]
    rlHypers = [2, 0.4]  # [k, scale(theta)] # mean=k*theta
    rhoHypers = [0.5, 0.05, 3, 3]  # [mu0, sigma0sq, kappa0, nu0]
    
    kernelscaleHypers = [1.2, 0.2, 3, 3]  # [mu0, sigma0sq, kappa0, nu0]

    
    hypers = [k_x0Hypers, dPHypers, rlHypers, rhoHypers, kernelscaleHypers]
    return hypers






In [7]:
# specify stimulus

def run_parallel_ribbon(*parameters):
    sampsParams = np.array(parameters)
    outUn =  runManyRibbon(light, g,trueG,dataSS, sampsParams, batchsize, w, stim_kind='light')
    return outUn

In [ ]:

## number of samples
# if nr is to small error araises from taking best 1000 values etc.
nSamps0 = 40000
nSampsLate = 20000
nsave = 1000

# number of sample to update from
nupdate0 = 10 # 20
nupdate_late = 10 # 20
importance_factor0 = 1#0.2
importance_factor_late = 1#0.2


batchsize = 4

"""
switching fitting
"""

# choose nr of processes working in parallel
pr = 35

# number of runs
runs = 100


# arrays for saving results 
# best nsave are saved
outRSave = np.zeros((runs,9,nsave))
hypersSave = []#np.zeros((runs,5,2))


# prior parameters for ribbon params
cHyper = [0,0,0,0,1,1,0,0] # 1 if constant, 0 else
c2 = [0,0,0,0,dockMax,ribbonMax,0,0]
constants = np.stack((cHyper,c2))  

#hypers = hypers_old[-1]
hypers = makeHypersPrior() # not used for uniform priors
hypersSave.append(hypers)

for noRun in range(0,runs):
    if noRun ==0:
        nSamps = nSamps0
        nupdate = nupdate0
        importance_factor = importance_factor0
    else:
        nSamps = nSampsLate
        nupdate = nupdate_late
        importance_factor = importance_factor_late
    print()
    print('run',noRun)
    
    
    """
    run for params
    """

    # sample params 
    sampsStackR = list([])


    for j in range(0,pr):
        sampsStackR.append(drawSamps(constants,hypers,int(nSamps/pr)))
    
    print('finished sampling, now run simulations...')

    with multiprocessing.Pool(processes=pr) as pool:
        outUnStackR = pool.starmap(run_parallel_ribbon, sampsStackR)

    #temp3 = datetime.datetime.now()
    #tempdiff = temp3 - temp2
    #print('time used for ribbon fitting',tempdiff)


    #reshape and sort results
    outUnStackR = np.array(outUnStackR)
    outUnR = np.zeros((9,np.shape(outUnStackR)[0]*np.shape(outUnStackR)[2]))
    #outUn = np.zeros((6,np.shape(outUnStack)[0]*np.shape(outUnStack)[2]))

    for i in range(0, np.shape(outUnStackR)[0]):
        outUnR[:,i*np.shape(outUnStackR)[2]:i*np.shape(outUnStackR)[2]+np.shape(outUnStackR)[2]] = outUnStackR[i,:,:]

    # sorting:
    outR = outUnR[:,outUnR[-1,:].argsort()]

    # update parameters
    # using mean of best threshold samples 
    threshold = nupdate 
    params = np.mean(outR[:-1,:threshold], axis=1)
    print('paramsmean =',params)
    
    #pairPlotStuff_truevalues(outR[0,:100],outR[1,:100],outR[3,:100],outR[2,:100], trueK,
     #                   trueX0, trueRibbonLambda,trueDockP, markersize=30)
    
    """
    Prior updating for params
    """
    # for params
    hypers = makeHypers(hypers, outR, nupdate_raw=nupdate, importance_factor=importance_factor) 
    
    print('updated hypers', hypers)
    timer.toc()
    
    
    hypersSave.append(hypers)
    outRSave[noRun,:,:] = outR[:,:nsave]
    
# save data

filename = "save_synthetic_data_v6_1.pkl"


metaInfo = 'batchsize: {}, nSamps0: {}, nSampsLate: {}, nupdate0: {}, nupdate_late: {}, w_scale: {}, runs: {}\
            '.format(batchsize,nSamps0, nSampsLate, nupdate0, nupdate_late, w_scale, runs)

trueparams = [trueK,trueX0,trueDockP,trueRibbonLambda,dockMax,ribbonMax, truerho, truescale]

dict_data = dict({'hypers': hypersSave, 'samples': outRSave, 'metaInfo': metaInfo, 
                  'trueparams':trueparams, 'truedata':data})
with open(filename, "wb") as f:
    pickle.dump(dict_data, f)




Tic time: 2019-05-16 16:02:34.592026

run 0
finished sampling, now run simulations...
paramsmean = [19.42439349  0.77893946  0.19340494  0.37665935  7.         50.
  0.37199328  1.03415171]
updated hypers [[array([16.73170963,  0.69924247]), array([[1.41991161e+03, 7.33267727e+00],
       [7.33267727e+00, 3.37069769e-01]]), 14, 14], [0.21800379800536065, 0.020114711696004074, 13, 13], [5.766593531746782, 0.08], [0.4015332920107538, 0.023397261300326145, 13, 13], [1.0724243914043465, 0.08082717935846853, 13, 13]]
Total used time: 0:42:14.816183

run 1
finished sampling, now run simulations...
paramsmean = [18.32950817  0.79542985  0.15386216  0.39748622  7.         50.
  0.35083518  1.03489003]
updated hypers [[array([17.39745903,  0.73932055]), array([[1.49523835e+03, 8.40732917e+00],
       [8.40732917e+00, 3.97502558e-01]]), 24, 24], [0.19011613078817127, 0.014109900673819115, 23, 23], [9.741455723448528, 0.044444444444444446], [0.3794906337453572, 0.02024892932217145, 23, 23], [1.05